In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Creating PICRUSt2 file (only needs to run once)

In [2]:
# Read the CSV files
EC_gibbs_info = pd.read_csv('PICRUSt2/GIBBs.EC.numbers.csv')
EC_gibbs = pd.read_csv('PICRUSt2/EMU_database.GIBBs.EC.PICRUSt2.R.csv')

# Set row names
EC_gibbs = EC_gibbs.set_index('tax_id')

In [3]:
EC_gibbs.head()

,Kingdom,Phylum,Class,Order,Family,Genus,Species,16S_rRNA_Count,metadata_NSTI,gc,...,EC.1.14.18.3,EC.1.14.99.39,EC.1.7.2.6,EC.1.7.1.15,EC.1.7.2.2,EC.1.7.1.4,EC.1.7.7.1,EC.1.7.2.1,EC.1.7.2.5,EC.1.7.2.4
tax_id,,,,,,,,,,,,,,,,,,,,,
100,Bacteria,Proteobacteria,Alphaproteobacteria,Hyphomicrobiales,Xanthobacteraceae,Ancylobacter,Ancylobacter aquaticus,2.0,0.009404,0.559407,...,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0
100053,Bacteria,Spirochaetes,Spirochaetia,Leptospirales,Leptospiraceae,Leptospira,Leptospira alexanderi,1.0,0.025089,0.528302,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000566,Bacteria,Actinobacteria,Actinomycetia,Pseudonocardiales,Pseudonocardiaceae,Halosaccharopolyspora,Halosaccharopolyspora lacisalsi,1.0,0.030423,0.597183,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,Bacteria,Bacteroidetes,Cytophagia,Cytophagales,Berrdetiaceae,Garritya,Garritya polymorpha,1.0,0.402240,0.497630,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001240,Bacteria,Actinobacteria,Actinomycetia,Micrococcales,Microbacteriaceae,Cryobacterium,Cryobacterium roopkundense,1.0,0.016080,0.562714,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Read another CSV file
df = pd.read_csv('rel_abund_16S_ABC_qpcr_adj.csv')

# Copying the dataframe
EC_sub = EC_gibbs.copy()

# Binarizing the numeric columns
EC_bin = EC_sub.apply(lambda x: (x > 0).astype(int) if np.issubdtype(x.dtype, np.number) else x)

# Get the sample names and enzyme names
samples = df.columns[9:]
enzymes = EC_sub.columns[11:]

# Calculate the column sums
reads = df[samples].sum(axis=0, skipna=True)

# Initialize the final dataframe
final = pd.DataFrame(index=enzymes, columns=samples)

In [5]:
# Nested loop for calculation
for i in samples:  # Limiting to first 3 samples
    for j in enzymes:
        # Calculation without the skipna parameter
        value = np.sum(df[i] / EC_bin['16S_rRNA_Count'] * EC_bin[j]) / reads[i]
        final.at[j, i] = value

In [24]:
# If you want to save 'final' to a CSV file
final.to_csv('PICRUSt2/GIBBs.rel_abund.final.A.csv')

In [25]:
final.head()

,barcode02A,barcode03A,barcode04A,barcode05A,barcode06A,barcode07A,barcode08A,barcode09A,barcode10A,barcode11A,...,barcode74C,barcode75C,barcode76C,barcode77C,barcode78C,barcode79C,barcode80C,barcode81C,barcode82C,barcode83C
EC.4.1.1.5,0.070501,0.032222,0.038203,0.069625,0.061416,0.052594,0.070817,0.029874,0.027948,0.058781,...,0.053962,0.048658,0.049178,0.047255,0.058373,0.049791,0.046048,0.059305,0.073815,0.066628
EC.1.1.1.76,0.094335,0.094093,0.060159,0.113105,0.086322,0.07846,0.117633,0.075767,0.099996,0.100183,...,0.069324,0.063201,0.0675,0.066325,0.065032,0.061272,0.055558,0.070769,0.091271,0.091634
EC.3.2.1.6,0.00258,0.0,0.003189,0.008596,0.001846,0.0,0.0,0.0,0.0,0.002505,...,0.002026,0.001127,0.000866,0.00153,0.000426,0.001286,0.001494,0.001252,0.001364,0.002906
EC.3.2.1.14,0.17726,0.217941,0.131829,0.198505,0.172831,0.128928,0.190086,0.157443,0.143782,0.223027,...,0.127043,0.127601,0.111586,0.139139,0.146757,0.117479,0.107287,0.124374,0.137074,0.13714
EC.2.6.1.86,0.002902,0.009929,0.004662,0.008633,0.00826,0.0,0.005652,0.0,0.0,0.00501,...,0.005069,0.005806,0.006803,0.005564,0.012218,0.003162,0.004271,0.002479,0.003745,0.006964


# Load PICRUSt2 file and mapping file

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load the datasets
gibbs_df = pd.read_csv('PICRUSt2/GIBBs.rel_abund.final.A.csv', index_col = 0)
mapping_df = pd.read_csv('mapping_file_with_metadata_harvest_biomass_2022.csv')

In [5]:
# Transpose the filtered GIBBS dataframe to have EC numbers as columns
gibbs_df = gibbs_df.transpose()

# Merge the GIBBS data with the mapping data on the Barcode_key column
merged_df = pd.merge(mapping_df, gibbs_df, left_on='Barcode_key', right_index=True)

In [6]:
merged_df.to_csv('Data/PICRUSt2/picrust_mapping_file.csv')
merged_df.head()

,Unnamed: 0,Sample name,Sample date,Sample ID,Plot,ID,Nitrogen,Water,Block,Direction,...,EC.1.14.18.3,EC.1.14.99.39,EC.1.7.2.6,EC.1.7.1.15,EC.1.7.2.2,EC.1.7.1.4,EC.1.7.7.1,EC.1.7.2.1,EC.1.7.2.5,EC.1.7.2.4
0,0,193,2021-06-15,2N,2N,111.0,Low,High,1,N,...,0.011413,0.011413,0.001463,0.255915,0.146247,0.0,0.019078,0.070940,0.097735,0.044308
1,1,205,2021-06-15,2S,2S,111.0,Low,High,1,S,...,0.010296,0.010296,0.001793,0.278651,0.138524,0.0,0.018311,0.076365,0.104344,0.037114
2,2,212,2021-06-15,23S,23S,211.0,Low,High,2,S,...,0.009314,0.009314,0.001190,0.339663,0.110387,0.0,0.006431,0.063950,0.091039,0.024709
3,3,221,2021-06-15,23N,23N,211.0,Low,High,2,N,...,0.009552,0.009552,0.000000,0.269004,0.133448,0.0,0.014147,0.074577,0.104856,0.044248
4,4,195,2021-06-15,32N,32N,311.0,Low,High,3,N,...,0.006488,0.006488,0.001442,0.316970,0.131443,0.0,0.016739,0.063306,0.078908,0.026063
